In [3]:
import re

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import spacy
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
from tqdm import tqdm

In [14]:
nlp = spacy.load("en")

In [5]:
tweets_train = pd.read_csv("/home/malits/data/twitter_sentiment_analysis/training.1600000.processed.noemoticon.csv", 
                           encoding = "ISO-8859-1")

In [6]:
tweets_train.head(3)

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire


### Create training data

In [33]:
training_data = pd.DataFrame()
training_data["sentiment"] = tweets_train["0"].apply(lambda v: 1 if (v > 2) else 0)
training_data["tweet"] = tweets_train.iloc[:, [5]]
training_data["tweet"] = training_data["tweet"].astype(str)

In [34]:
training_data.head(4)

,sentiment,tweet
0,0,is upset that he can't update his Facebook by ...
1,0,@Kenichan I dived many times for the ball. Man...
2,0,my whole body feels itchy and like its on fire
3,0,"@nationwideclass no, it's not behaving at all...."


In [35]:
#@Source https://www.analyticsvidhya.com/blog/2018/07/hands-on-sentiment-analysis-dataset-python/

def remove_handles(text):
    r = re.findall(r'@[\w]', text)
    
    for i in r:
        re.sub(i, '', text)
        
    return text

In [9]:
extra_char_regex = re.compile(r'[^A-Za-z#]')

training_data["clean_tweet"] = np.vectorize(remove_handles)(training_data["tweet"])
training_data["clean_tweet"] = training_data["clean_tweet"].str.replace(extra_char_regex, " ")


Once preprocessed, split to test/train sets

In [32]:
X, y = training_data["clean_tweet"], training_data["sentiment"]

X_train, X_test, y_train, y_test = train_test_split(X, y)

In [22]:
def lemmatize_sentence(text):
    doc = nlp(text)
    
    text = " ".join([t.lemma_ for t in doc])
    
    return text

### Tokenizing for embedding

In [26]:
#@Source https://towardsdatascience.com/machine-learning-word-embedding-sentiment-classification-using-keras-b83c28087456

tokenizer = Tokenizer()
total_reviews = X_train + X_test
tokenizer.fit_on_texts(total_reviews)

max_length = max([len(s.split()) for s in total_reviews])
vocab_size = len(tokenizer_obj.word_index) + 1

X_train_tokens = tokenizer.texts_to_sequences(X_train)
X_test_tokens = tokenizer.texts_to_sequences(X_test)

X_train_pad = pad_sequences(X_train_tokens, maxlen=max_length, padding='post')
X_test_pad = pad_sequences(X_test_tokens, maxlen=max_length, padding='post')


AttributeError: 'float' object has no attribute 'lower'